<a href="https://colab.research.google.com/github/alvesmgabriel/intro-opencv/blob/main/topico-02-transformacoes-geometricas/topico_02_transformacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformações geométricas em imagens digitais

Neste tópico veremos as **Transformações geométricas**. Inicialmente iremos importar as bibliotecas necessárias para realizar tais operações e utilizaremos, como teste, a imagem da *Lenna* que possui dimensões `512 x 512` pixels.

> **Curiosidade**: A imagem da Lenna é amplamente utilizada na área de Processamento de Imagens desde a década de 70. Para saber mais informações sobre a origem desta imagem consulte o [Wikipedia](https://en.wikipedia.org/wiki/Lenna) ou o site [The Lenna Story](http://www.lenna.org/).


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# local em que a imagem está armazenada
caminho_imagem = '/content/drive/MyDrive/Colab Notebooks/topico-02-transformacoes/dados/lenna.png'

# carrega a imagem
lenna = cv2.imread( caminho_imagem )

Pronto. Depois de carregar a imagem, caso queira verificar, pode exibi-la para se certificar que tudo correu bem. Importante mencionar que a variável `caminho_imagem` possibilita que o local da imagem seja alterado se necessário.

In [ ]:
cv2_imshow( lenna )
print('\nA imagem possui {0} pixels'.format(lenna.shape[:2]))

## Transformações geométricas

Para efeitos de estudo das tranformações geométricas, selecionaremos um trecho da imagem com dimensão `192 x 192` pixels iniciando na posição `(linha=200, coluna=190)` de modo que o rosto da Lenna seja selecionado.

> **Você sabia?** Ao selecionar um trecho da imagem, ou seja, "*cortá-la*" estamos realizando a transformação geométrica conhecida como **cropping**.

In [ ]:
rosto_lenna = lenna[200:392, 190:382, :]
cv2_imshow( rosto_lenna )

O motivo da seleção é trabalhar com uma matriz menor de pixels. Estudaremos as principais transformações geométricas: *scaling*, *translating*, *rotating*, *cropping*, *flip*, *perspective* e *affine*.

> **Importante**: Ao utilizarmos o recurso de [*slicing Numpy*](https://www.w3schools.com/python/numpy/numpy_array_slicing.asp) realizamos a transformação geométrica conhecida como *cropping*.


### Scaling

O processo de *scaling* corresponde a ampliar, ou reduzir, as dimensões da imagem e, portanto, modificá-la. Essa modificação pode-se dá por meio de um *fator*, daí geralmente chama-se <u>scaling</u>; ou se especifica o novo tamanho da imagem, neste caso às vezes o processo é chamado de <u>sizing</u> ou <u>resizing</u>.

> **Nota**: na prática utilizar fator de alteração ou definir o novo tamanho corresponderá a transformação geométrica conhecida como *scaling*.

Na biblioteca OpenCV a função `resize()` realiza esta transformação geométrica. A seguir será ilustrado o processo utilizando o fator de alteração bem como a definição de um novo tamanho.

In [ ]:
altura, largura = rosto_lenna.shape[:2]
print('altura = {0}; largura = {1}'.format(altura, largura))

Primeiro, obtemos a largura e a altura da matriz que iremos trabalhar e agora vamos definir um novo tamanho para a imagem de saida. Perceba que é necessário realizar a interpolação.

No caso da ampliação, a nova matriz conterá *mais* pixels cujos valores precisarão ser determinados em função dos valores dos pixels já existentes. No caso da redução, a nova matriz conterá *menos* pixels e precisará descartar pixels já existentes. Em ambos os casos a interpolação busca preservar a informação original e minimizar erros. O OpenCV fornece cinco métodos de interpolação.

> **Nota**: No caso da redução é possível realizar a divisão, por exemplo: `largura // 4`. Perceba, no entanto, o operador `//`, da linguagem Python, que retorna a *divisão inteira*. Isso é necessário, pois as dimensões são expressas em números inteiros e não ponto-flutuantes.



In [ ]:
rosto_ampliado4x = cv2.resize(rosto_lenna, (largura * 2, altura * 2), interpolation=cv2.INTER_CUBIC)
cv2_imshow( rosto_ampliado4x )

O próximo exemplo mostra o *scaling* utilizando agora um fator de alteração. Ao utilizar o *fator* é possível passar um valor de ponto-flutuante.

In [ ]:
rosto_fator = cv2.resize(rosto_lenna, None, fx=0.5, fy=0.5)
cv2_imshow( rosto_fator )

### Translação

A translação (*translating*) consiste em deslocar a imagem um determinado número de linhas e colunas. Esse processo é realizado pela função `warpAffine()` que utiliza <u>coordenadas homôgeneas</u> e uma matriz de transformação `M = 2 x 3`. Portanto, o resultado na posição `(l, c)` ($R_{l,c}$) da translação de um determinado ponto da imagem pode ser obtida por meio da seguinte equação:

$$
R_{l, c} = 
\begin{bmatrix}
1 & 0 & off\_lin \\
0 & 1 & off\_col
\end{bmatrix}
\begin{bmatrix}
l \\
c \\
1
\end{bmatrix}
$$

onde `off_col` e `off_lin` indicam, respectivamente, o deslocamento da largura (colunas) e altura (linhas) e `l` e `c` determina a posição original `(linha, coluna)` que será deslocada.

A título de exemplo, considere o ponto com coordenadas cartesianas `(0,0)` e o deslocamento de 20 linhas e 10 colunas. Logo, a nova coordenada será:
$$
R_{0, 0} = 
\begin{bmatrix}
1 & 0 & 20 \\
0 & 1 & 10
\end{bmatrix}
\begin{bmatrix}
0 \\
0 \\
1
\end{bmatrix} = (20, 10)
$$





In [ ]:
import numpy as np
offset_linhas = 20
offset_colunas = 10
M = np.float32([[1, 0, offset_colunas], [0, 1, offset_linhas]])
dest = cv2.warpAffine(rosto_lenna, M, (largura, altura))
cv2_imshow( dest )

No exemplo acima, deslocamos 20 linhas e 10 colunas na imagem original. O fato da coordenada inicial `(0,0)` encontra-se no canto superior esquerdo justifica as faixas escuras na parte superior e na lateral esquerda. Isto porquê não há alteração nas dimensões originais da imagem, pois se trata de um *movimento rígido*.

### Rotação

Essa transformação consiste em rotacionar os pixels de uma imagem em um determinado ângulo, tanto no sentido horário quanto anti-horário. Para isso é utilizada a matriz de transformação `2 x 3` ($M$), conforme indicado abaixo.

$$
M = \begin{bmatrix}
\alpha & \beta & (1 - \alpha).c_l - \beta . c_a \\
-\beta & \alpha & \beta . c_l + (1 - \alpha) . c_a
\end{bmatrix}
$$

onde $c_l$ é o centro da imagem no sentido da largura e $c_a$ é o centro da imagem no sentido da altura. O valor de $\alpha$ é dado por $\alpha = escala . \cos\theta$ e $\beta$ dado por $\beta = escala . \sin\theta$. Na definição de $\alpha$ e $\beta$, temos $\theta$ que corresponde ao ângulo de rotação e a $escala$ que corresponde a um fator de ampliação ou redução da imagem.

A biblioteca OpenCV disponibiliza a função `getRotationMatrix2D()` que nos auxilia na definição da matriz $M$. Para isso são necessários três parâmetos: o centro da imagem; o ângulo de rotação ($\theta$) e a $escala$. Em seguida, utilizamos novamente a função `warpAffine()` para realizar a rotação.

> **Importante**: valores positivos correspondem a rotação no sentido anti-horário.

In [ ]:
M = cv2.getRotationMatrix2D((largura/2.0, altura/2.0), 0, 1)
dst = cv2.warpAffine(rosto_lenna, M, (largura, altura))
cv2_imshow(dst)
M

### Flip

Essa operação consiste em "espelhar" a imagem. A biblioteca OpenCV disponibiliza a função `flip()` para realizar tal operação. Além da imagem, a função possui outro parâmetro que informa o tipo do "flip":
- 0, indica que o *flip* será na vertical;
- 1, indica que o *flip* será na horizontal;
- -1, indica que o *flip* será em ambas as direções.

In [ ]:
im_flip = cv2.flip(rosto_lenna, -1)
cv2_imshow( im_flip )